## Maximum likelihood model fitting

In [1]:
import numpy as np
import os 
import arviz as az
import scipy as sp
import scipy.io as sio
import scipy.stats as stats
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
import importlib
import math

In [2]:
os.chdir('..\\Pystan\\nc_files')

In [3]:
standard_basic_fit = az.from_netcdf('standard_basic_fit.nc')
uncued_basic_fit = az.from_netcdf('uncued_basic_fit.nc')

In [4]:
standard_basic_fit

Inference data with groups:
	> posterior
	> sample_stats

In [5]:
standard_basic_fit.posterior.beta #beta's DataArray 
# standard_basic_fit.posterior.beta.beta_dim_0 #beta's subjects
standard_basic_fit.posterior.beta[0] #first chain, storing an array of beta values
chain_1_beta = standard_basic_fit.posterior.beta[0] #first chain, each column represents a subject; each column stores the posterior values of beta
chain_1_beta_df = pd.DataFrame(data = chain_1_beta) #chain_1 as a df
chain_1_beta[:,0] #first column of the first chain, representing the beta values of the posterior 

<xarray.DataArray 'beta' (draw: 1000)>
array([2.52938 , 2.873311, 2.229288, ..., 2.097382, 2.378004, 2.430716])
Coordinates:
    chain       int64 0
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
    beta_dim_0  int64 0

In [6]:
chain_1_beta

<xarray.DataArray 'beta' (draw: 1000, beta_dim_0: 32)>
array([[2.52938 , 6.647068, 3.446754, ..., 5.800199, 2.519514, 3.995434],
       [2.873311, 4.390683, 3.429956, ..., 2.978015, 2.220725, 5.540741],
       [2.229288, 5.877074, 4.153091, ..., 3.554596, 1.695332, 5.486356],
       ...,
       [2.097382, 4.577933, 3.651047, ..., 3.893158, 2.880784, 3.750539],
       [2.378004, 5.84317 , 2.689029, ..., 4.813331, 1.838248, 4.658764],
       [2.430716, 5.72351 , 3.396499, ..., 4.287735, 2.098062, 4.562273]])
Coordinates:
    chain       int64 0
  * draw        (draw) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
  * beta_dim_0  (beta_dim_0) int64 0 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31

In [7]:
standard_basic_fit.posterior.etaPositive #etaPositive's DataArray
standard_basic_fit.posterior.etaPositive[0] #first chain
chain_1_etaP = standard_basic_fit.posterior.etaPositive[0]

standard_basic_fit.posterior.etaNegative #etaNegative's DataArray
standard_basic_fit.posterior.etaNegative[0] #first chain
chain_1_etaN = standard_basic_fit.posterior.etaNegative[0]

standard_basic_fit.posterior.m #m's DataArray 
standard_basic_fit.posterior.m[0] #first chain
chain_1_m = standard_basic_fit.posterior.m[0]

In [8]:
chain_1_etaP
chain_1_etaN
chain_1_m

<xarray.DataArray 'm' (draw: 1000, m_dim_0: 32)>
array([[ 0.01708 ,  0.002918,  0.054323, ..., -0.009634,  0.004339,  0.027044],
       [ 0.009775,  0.055069,  0.020398, ...,  0.022023,  0.032587,  0.032938],
       [ 0.020816,  0.045591,  0.015153, ..., -0.002321,  0.023191,  0.036932],
       ...,
       [ 0.029665,  0.03129 ,  0.044791, ..., -0.001322,  0.000159,  0.053835],
       [ 0.023606,  0.033369,  0.07561 , ..., -0.003759,  0.031682,  0.058283],
       [ 0.014269,  0.053067,  0.055448, ..., -0.007403,  0.024158,  0.050805]])
Coordinates:
    chain    int64 0
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
  * m_dim_0  (m_dim_0) int64 0 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30 31

In [9]:
chain_1_etaP[:,[0]]
chain_1_etaN[:,[0]]
chain_1_m[:,[0]]

<xarray.DataArray 'm' (draw: 1000, m_dim_0: 1)>
array([[0.01708 ],
       [0.009775],
       [0.020816],
       ...,
       [0.029665],
       [0.023606],
       [0.014269]])
Coordinates:
    chain    int64 0
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
  * m_dim_0  (m_dim_0) int64 0

## Goal: create params for one subject (params_s)

params_s will be 4 columns (representing beta, etaP, etaN and m) and 1000 rows (representing the 1000 draws)

## Goal 2: write for loop such that the code can run multiple times, using a different subject each time ***
- This will involve taking the first column from each parameter's dataarray (for subject 0), appending them (side-by-side), then running the code

In [ ]:
array1 = np.array([[1, 1, 1], [2, 2, 2]])
array2 = np.array([[3, 3, 3], [4, 4, 4]])

appended = np.concatenate([array1, array2], axis = 1)
appended

In [ ]:
array1 = np.array([[1,2,3]])
array2 = np.array([[4,5,6]])

appended = np.concatenate([array1, array2], axis = 0)
# appended

In [ ]:
parameters = np.concatenate([chain_1_beta[:,[0,1]], chain_1_etaP[:,[0,1]], chain_1_etaN[:,[0,1]]], axis = 1)
# parameters #a workaround, but contains 2 beta values, 2 etaP, and 2etaN (as the 6 columns)

In [10]:
params_1 = np.concatenate([chain_1_beta[:,[0]], chain_1_etaP[:,[0]], chain_1_etaN[:,[0]], chain_1_m[:,[0]]], axis = 1) #extra square parentheses add a dimension
params_1

array([[2.52937956, 0.02981436, 0.11405636, 0.01707991],
       [2.87331074, 0.02574753, 0.12769834, 0.00977511],
       [2.22928782, 0.04103009, 0.12970154, 0.02081641],
       ...,
       [2.09738164, 0.03277643, 0.12669542, 0.02966539],
       [2.3780043 , 0.04014199, 0.16419756, 0.02360568],
       [2.43071564, 0.03870608, 0.12297809, 0.01426914]])

In [11]:
def get_params_basic(fit, s):
    """takes in fit, s (subject number), and outputs params_s using the first chain
    function is called basic due to the parameters chosen"""
    params_s = np.concatenate([fit.posterior.beta[0][:,[s-1]], 
                               fit.posterior.etaPositive[0][:,[s-1]], 
                               fit.posterior.etaNegative[0][:,[s-1]]], 
                              axis = 1)
    return params_s

#fit.posterior.m[0][:,[s-1]]

In [12]:
params_1 = get_params_basic(standard_basic_fit, 1)
params_1

array([[2.52937956, 0.02981436, 0.11405636],
       [2.87331074, 0.02574753, 0.12769834],
       [2.22928782, 0.04103009, 0.12970154],
       ...,
       [2.09738164, 0.03277643, 0.12669542],
       [2.3780043 , 0.04014199, 0.16419756],
       [2.43071564, 0.03870608, 0.12297809]])

In [ ]:
def get_params(fit, subs): 
    """gets params for subs, this function is not functional yet"""
    for s in range(subs):
        params = np.concatenate([fit.posterior.beta[:,[s]], fit.posterior.etaPositive[:,[s]], fit.posterior.etaNegative[:,[s]]], axis = 1)
        #... rest of body 
    return params

In [ ]:
# get_params(standard_basic_fit, 1)

# Original simulation code

In [ ]:
np.random.seed(
ntr = 200 # num trials #might be 1000

def mysimulation(params,ntr):
    
    # params is a 3 vector of beta, etaP, etaN
    
    V = np.zeros(4) # each option has a value
    beta = params[0]
    etaP = params[1]
    etaN = params[2]
    decay = params[3] # decay between zero and 1
    
#     V[3] = 4
    
    p_win = [0.9,0.8,0.5,0.4]
    win_amount = [1,2,3,4]
    pun_dur = [5,10,30,40]
    
    Q = np.zeros([4,ntr])
    choice = []
    win = []
    probs = []
    
    
    for t in range(ntr):
#         print(t)

        Q[:,t] = V
        
        # now we want to calculate the log likelihood of the choice on the current trial
        # we assume the prob of each choice follows a softmax rule
        # in log this looks like this
        
        p_action = np.exp(beta*V)/np.sum(np.exp(beta*V))
        
        # pick an action according to these probabilities
        chosen = np.random.choice([0,1,2,3], size=None, replace=True, p=p_action)
    
        # now we want to learn from feedback on this trial
        # win or lose?
        outcome = np.random.choice([1,0], size=None, replace=True, p=[p_win[chosen], 1-p_win[chosen]])
        
        if outcome:
            V[chosen] += etaP*(win_amount[chosen] - V[chosen])
            # this is the same as writing
            # V[chosen option] = V[chosen option] + eta*(reward - V[chosen option])
        else:
            V[chosen] += etaN*(-pun_dur[chosen] - V[chosen])
        
        # values decay with time if unchosen
        ind = np.setdiff1d([0,1,2,3],chosen)
#         print(ind)
        V[ind] = decay*V[ind]
        
        choice.append(chosen)
        win.append(outcome)
        probs.append(p_action)
        
    return Q,choice,win,probs

# New Simulation Code

In [ ]:
# import random
# random.seed(10)

In [ ]:
# params is a 3 vector of beta, etaP, etaN
params = [2.52937956, 0.02981436, 0.11405636]
ntr = 1000

V = np.zeros(4) # [0,0,0,0]
beta = params[0]
etaP = params[1]
etaN = params[2]
# decay = params[3] # decay between zero and 1
    
p_win = [0.9,0.8,0.5,0.4]
win_amount = [1,2,3,4]
pun_dur = [5,10,30,40]

Q = np.zeros([4,ntr]) #1 array of 4 rows and 1000 columns
choice = []
win = []
probs = []

In [ ]:
Q
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])
arr[2,:] = V
arr

In [ ]:
for t in range(ntr): #from 0 to 999

    Q[:,t] = V #each column of Q (which represents a t) is now equal to V (4 zeros). This essentially makes each column equal to 4 rows of zero

    p_action = np.exp(beta*V)/np.sum(np.exp(beta*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

    chosen = np.random.choice([0,1,2,3], size=None, replace=True, p=p_action) #a simulated sample of choices (0-3 rep: P1-P4) using the probabilities from p_action
#     print(chosen)
    outcome = np.random.choice([1,0], size=None, replace=True, p=[p_win[chosen], 1-p_win[chosen]]) #a simulated sample of outcomes based on the choice made (in chosen)

    if outcome: #if outcome == 1:
        V[chosen] += etaP*(win_amount[chosen] - V[chosen])
        # this is the same as writing
        # V[chosen option] = V[chosen option] + eta*(reward - V[chosen option])
    else: #if outcome == 0:
        V[chosen] += etaN*(-pun_dur[chosen] - V[chosen])

    choice.append(chosen)
    win.append(outcome)
    probs.append(p_action)

# return Q,choice,win,probs

In [ ]:
len(probs) 

## Goal 3: we now need to acquire actual chosen and outcome data, and ntr... for a certain subject 

In [16]:
os.chdir("C:\\Users\\dexte\\sparklyRGT\\sparklyRGT_tutorial") 
import sparklyRGT as rgt
os.chdir("C:\\Users\\dexte\\sparklyRGT\\Pystan") 
import model_data as md

I am being executed!


In [17]:
fnames = ['BH09_raw-free_S1-5_corrected.xlsx','CH02_corrected.xlsx','NA01_raw_free-choice_S8-18.xlsx',"CH01_corrected.xlsx"]
df = rgt.load_multiple_data(fnames, reset_sessions = True)

In [18]:
#rename MSNs so that the rats on the outcome task don't have "loss" in the MSN
for i in range(len(df)):
    if df.at[i, 'MSN'] == 'LossrGT_A-losscue_v1':
        df.at[i,'MSN'] = 'outcomeRGT_A'
    if df.at[i, 'MSN'] == 'LossrGT_B-losscue_v1':
        df.at[i,'MSN'] = 'outcomeRGT_B'
        
#rename MSNs so that the rats on the random task don't have "loss" in the MSN
for i in range(len(df)):
    if df.at[i,'MSN'] == 'AnarchyrGT_B-losscue_v6':
        df.at[i,'MSN'] = 'RandomRGT_B'
    if df.at[i,'MSN'] == 'AnarchyrGT_A-losscue_v6':
        df.at[i,'MSN'] = 'RandomRGT_A'
        
        
task_list = df.groupby(['MSN'])['Subject'].unique()

In [19]:
uncued_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Classic' in task]])
standard_subs = np.concatenate((task_list['rGT_A-cue'], task_list['rGT_B-cue']))
#concatenating together MisRGT tasks, and RevRGT tasks, as they both refer to reverse-cue RGT
reverse_subs = np.concatenate((np.concatenate(task_list[[task for task in df.MSN.unique() if 'Mis' in task]]),
                              np.concatenate(task_list[[task for task in df.MSN.unique() if 'Rev' in task]])))
outcome_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'outcome' in task]])
random_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Random' in task]])
loss_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'oss' in task]])

subs = [uncued_subs,standard_subs,reverse_subs,outcome_subs,random_subs,loss_subs]

In [20]:
uncued_subs

array([202, 218, 222, 310, 314, 401, 403, 405, 407, 415, 204, 220, 224,
       309, 313, 402, 404, 406, 408, 416, 325, 329, 326, 330], dtype=int64)

In [21]:
numsessions = 5 #***first 5 sessions?
uncued = md.get_model_data(df, numsessions, subs[0])

In [22]:
# uncued

In [23]:
uncued_df = pd.DataFrame.from_dict(uncued)
uncued_df 
# C (chosen hole of 5), R (reward in pellets, 0 if loss), P (pun_dur, >0 if loss), O (P1-P4 option)

,N,ntr,startSubject,startSession,C,R,P,O
0,24,14273,1,1,1,1,0,1
1,24,14273,0,0,1,1,0,1
2,24,14273,0,0,2,4,0,4
3,24,14273,0,0,4,2,0,2
4,24,14273,0,0,1,0,5,1
...,...,...,...,...,...,...,...,...
14268,24,14273,0,0,0,0,0,0
14269,24,14273,0,0,5,2,0,2
14270,24,14273,0,0,0,0,0,0
14271,24,14273,0,0,5,2,0,2


In [24]:
subjects = uncued_df.startSubject.unique()
np.delete(subjects, 1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int64)

In [25]:
uncued_df = uncued_df.drop(uncued_df[(uncued_df.C == 0) & (uncued_df.startSubject == 0)].index)
uncued_df.reset_index(inplace = True)
uncued_df.index

RangeIndex(start=0, stop=10458, step=1)

In [26]:
for row in uncued_df.index:
    if uncued_df.at[row,'R'] == 0: #loss
        uncued_df.at[row,'outcome'] = 0
    if uncued_df.at[row,'P'] == 0: #win
        uncued_df.at[row,'outcome'] = 1
    
uncued_df

,index,N,ntr,startSubject,startSession,C,R,P,O,outcome
0,0,24,14273,1,1,1,1,0,1,1.0
1,1,24,14273,0,0,1,1,0,1,1.0
2,2,24,14273,0,0,2,4,0,4,1.0
3,3,24,14273,0,0,4,2,0,2,1.0
4,4,24,14273,0,0,1,0,5,1,0.0
...,...,...,...,...,...,...,...,...,...,...
10453,14262,24,14273,0,0,5,2,0,2,1.0
10454,14265,24,14273,0,0,5,2,0,2,1.0
10455,14266,24,14273,0,0,1,0,40,4,0.0
10456,14269,24,14273,0,0,5,2,0,2,1.0


In [27]:
uncued_df.index[uncued_df['startSubject'] == 2].tolist()
value = uncued_df.index[uncued_df['startSubject'] == 2]
value

Int64Index([509], dtype='int64')

In [28]:
uncued_1 = uncued_df[0:509] #df for subject 1 
uncued_1[['O', 'outcome']]
chosen_data = uncued_1['O']
outcome_data = uncued_1['outcome']
outcome_data[0]

1.0

## Goal 4: after acquiring chosen_data (option, O) and outcome_data (0 or 1 depending on R and P), we now run the model on the data to acquire p_action for each trial

In [ ]:
ntr = 509
for t in range(ntr): #from 0 to 508

    Q[:,t] = V #each column of Q (which represents a t) is now equal to V (4 zeros). This essentially makes each column equal to 4 rows of zero

    p_action = np.exp(beta*V)/np.sum(np.exp(beta*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

    if outcome_data[t] == 1: #if outcome == 1:
        V[chosen_data[t]-1] += etaP*(win_amount[chosen_data[t]-1] - V[chosen_data[t]-1]) #subtract 1 because chosen_data stores P1-P4 as 1-4 instead 0-3
    else: #if outcome_data == 0:
        V[chosen_data[t]-1] += etaN*(-pun_dur[chosen_data[t]-1] - V[chosen_data[t]-1])

    probs.append(p_action)

In [ ]:
#probs

## Goal 5: we now want the log(p_action) for the option chosen. 

ex. p_action = [0.245, 0.28, 0.235, 0.24] ... and P4 was then chosen. We would take log(0.24) for this trial 

In [ ]:
len(probs)
# probs #list of 509 arrays

In [ ]:
len(chosen_data)
# chosen_data #series object of 509 choices (1-4 representing P1-P4)

In [ ]:
import math
math.log10(10)
probs[0][0]
math.log10(probs[0][0]) #first array, first value
log_lik = pd.DataFrame()
log_lik
math.log10(probs[0][chosen_data[1]-1])

In [ ]:
d = {'col1': [0]}
log_lik_test = pd.DataFrame(d)
log_lik_test
log_lik_test[f"log_lik[{0}]"] = math.log10(probs[0][chosen_data[1]-1])
log_lik_test
log_lik = pd.DataFrame(d)

# log_lik_test.assign(log_lik_test=[92,81,66])

In [ ]:
for t in range(len(probs)): 
    log_lik[f"log_lik[{t}]"] = math.log10(probs[t][chosen_data[t]-1])
    
log_lik['sum'] = log_lik.sum(axis = 1)
log_lik

## Goal 6: we now want each column (ex. log_lik[5]) to contain 1000 values (each value is a log likelihood) 

Procedure: 
- Figure out how to add a value to the next row of the same column (ex. add the next value to log_lik[0] as another row)
    - either add a list as a new row
    - or, add a single value to the bottom of a column
- Form list comprehension of trials and params 

In [ ]:
d = {'col1': np.zeros(3)}
log_lik_test = pd.DataFrame(d)
log_lik_test.loc[log_lik_test.index[2], 'col1'] = 4
# df.loc[df.index[someRowNumber], 'New Column Title'] = "some value"
log_lik_test

In [30]:
ntr = 509
trialXparams = [(t, p) for t in range(ntr) for p in range(1000)]
# trialXparams

paramsXtrial = [(p, t) for p in range(1000) for t in range(ntr)]
# paramsXtrial

# for (p, t) in paramsXtrial: #one set of parameters, for all the trials, then the next set of parameters (params[1]) for all the trials, etc. 
#     print(p, t)

## Final goal: summarize the code (skip goal 4 and 5) 

In [31]:
# params is a 3 vector of beta, etaP, etaN
import math
params = params_1 #1000 sets of parameters

V = np.zeros(4) # [0,0,0,0]
# decay = params[3] # decay between zero and 1
    
p_win = [0.9,0.8,0.5,0.4]
win_amount = [1,2,3,4]
pun_dur = [5,10,30,40]
ntr = 509

Q = np.zeros([4,ntr]) #1 array of 4 rows and ntr columns
probs = []

d = {'col1': np.zeros(1000)}
log_lik = pd.DataFrame(d)
for (p, t) in paramsXtrial: #from 0 to 508

    Q[:,t] = V #each column of Q (which represents a t) is now equal to V (4 zeros). This essentially makes each column equal to 4 rows of zero
    
    if t == 0: #if using a new set of parameters...
        V = np.zeros(4) #reset the V values

    p_action = np.exp(params[p][0]*V)/np.sum(np.exp(params[p][0]*V)) #p_action holds the probabilities of each action (P1-P4), starting at 0.25 for each

    if outcome_data[t] == 1: #if outcome == 1:
        V[chosen_data[t]-1] += params[p][1]*(win_amount[chosen_data[t]-1] - V[chosen_data[t]-1]) #subtract 1 because chosen_data stores P1-P4 as 1-4 instead 0-3
    else: #if outcome_data == 0:
        V[chosen_data[t]-1] += params[p][2]*(-pun_dur[chosen_data[t]-1] - V[chosen_data[t]-1])
         
    log_lik.loc[log_lik.index[p], f"log_lik[{t}]"] = math.log10(p_action[chosen_data[t]-1])
    
    probs.append(p_action)

In [32]:
log_lik

,col1,log_lik[0],log_lik[1],log_lik[2],log_lik[3],log_lik[4],log_lik[5],log_lik[6],log_lik[7],log_lik[8],...,log_lik[499],log_lik[500],log_lik[501],log_lik[502],log_lik[503],log_lik[504],log_lik[505],log_lik[506],log_lik[507],log_lik[508]
0,0.0,-0.60206,-0.577731,-0.619112,-0.654395,-0.605261,-0.578005,-0.507950,-0.444523,-4.031998,...,-3.805040,-3.603531,-3.408052,-3.218434,-3.034518,-2.856155,-2.683208,-0.000124,-0.000110,-3.052252
1,0.0,-0.60206,-0.578189,-0.618807,-0.653370,-0.605056,-0.566462,-0.498414,-0.436567,-5.043489,...,-4.880271,-4.657008,-4.439497,-4.227590,-4.021145,-3.820024,-3.624096,-0.000011,-0.000010,-4.015127
2,0.0,-0.60206,-0.572613,-0.622860,-0.666332,-0.606915,-0.593047,-0.507537,-0.432094,-4.029111,...,-3.154439,-2.927646,-2.710292,-2.502058,-2.302667,-2.111891,-1.929560,-0.000581,-0.000509,-2.354966
3,0.0,-0.60206,-0.581579,-0.616227,-0.645543,-0.604531,-0.593622,-0.533624,-0.478812,-2.425131,...,-2.095204,-1.961679,-1.833277,-1.709924,-1.591565,-1.478166,-1.369713,-0.005242,-0.004852,-1.621858
4,0.0,-0.60206,-0.575877,-0.620463,-0.658680,-0.605821,-0.576646,-0.501481,-0.434135,-4.658627,...,-3.877874,-3.647812,-3.425576,-3.210916,-3.003596,-2.803400,-2.610130,-0.000112,-0.000096,-3.028983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,-0.60206,-0.577123,-0.619579,-0.655818,-0.605373,-0.565392,-0.494453,-0.430399,-5.601191,...,-4.836435,-4.593211,-4.356927,-4.127390,-3.904408,-3.687801,-3.477394,-0.000013,-0.000011,-3.902502
996,0.0,-0.60206,-0.581767,-0.616163,-0.645185,-0.604290,-0.566903,-0.508828,-0.455391,-3.988756,...,-4.010473,-3.829630,-3.653373,-3.481590,-3.314175,-3.151026,-2.992046,-0.000074,-0.000065,-3.312558
997,0.0,-0.60206,-0.579863,-0.617504,-0.649462,-0.604846,-0.575411,-0.511512,-0.453446,-3.749551,...,-3.230266,-3.043849,-2.863611,-2.689379,-2.520995,-2.358319,-2.201232,-0.000441,-0.000393,-2.542612
998,0.0,-0.60206,-0.571344,-0.623841,-0.669381,-0.607264,-0.584477,-0.496066,-0.418457,-5.337064,...,-3.965542,-3.691165,-3.427837,-3.175136,-2.932674,-2.700095,-2.477091,-0.000101,-0.000085,-2.979145


## Wrap-up: write function(s)
- remove col1 
- transform a sum column, or take the sum of the entire dataframe (log_lik) 

In [ ]:
# probs should be real probabilities

# Q,choice,win,probs = mysimulation([2,0.1,0.02,0.9],ntr)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=[12,5])

ax[0].plot(np.array(probs))
ax[0].legend(['1','2','3','4'])

ax[1].plot(np.arange(ntr),Q.transpose())

In [ ]:
plt.hist(choice)

In [ ]:
# let's check out what softmax does to choice probabilties for diff values

def softmax(x,beta):
    
    return np.exp(beta*x)/np.sum(np.exp(beta*x))

In [ ]:
plt.plot([10,40],softmax(np.array([10,40]),0),'*')
plt.plot([10,40],softmax(np.array([10,40]),0.5),'*')
plt.plot([10,40],softmax(np.array([10,40]),0.01),'*')

In [ ]:
log_liks = log_lik.drop('col1', axis=1)

In [ ]:
log_liks

In [ ]:
log_liks['row sum'] = log_liks.sum(axis = 1) #watch out! if you run this cell multiple times, it will continue adding

In [ ]:
pw_log_lik = log_liks['row sum'].sum()

In [ ]:
pw_log_lik

### Function (loglik_basic_sub) 

- parameters 
    - outcome_data
    - chosen_data 
    - params 
    - sub

- function template 
    - set variables (V, Q) 
    - set model (p_win, win_amount, pun_dur) 
    - 3 helper functions: 
        - get_outcome_data 
        - get_chosen_data
        - get_ntr_sub
    - run each ntr (should be a range from starting trial to end trial) X sub combination through the model 
    - output: sum of the log-likelihoods, for all subjects!
    
Workflow: 
- create df with subject, range(ntr), chosen_data, outcome_data as the columns 
- create df for each subject (by calling data in a row) 

In [58]:
def ntr_sub(df, numsessions, subs): 
    #load model data
    md_dict = md.get_model_data(df, numsessions, subs) #model data as a dict
    md_df = pd.DataFrame.from_dict(md_dict) #model data as a df
    md_df = md_df.drop(md_df[(md_df.C == 0) & (md_df.startSubject == 0)].index) #remove non-responses, while keeping changes in startSubject
    md_df.reset_index(inplace = True)
    
    subs_unique = md_df.startSubject.unique()
    subs_unique = np.delete(subs_unique, 1) #remove subject 0 (not a real subject)
    
    ntr_sub_ranges = []
    ntr_df = pd.DataFrame({'subject#': subs_unique})
    #get range for sub: 
    for s in subs_unique: 
        first_t = md_df.index[md_df['startSubject'] == s].tolist() #first trial
        last_t = md_df.index[md_df['startSubject'] == (s+1)].tolist() #last trial
        
        if last_t != []: #if s+1 does not exist (when s = final subject#), last_t = []
            ntr_sub = range(first_t[0], last_t[0])
            ntr_sub_ranges.append(ntr_sub)
        elif last_t == []:
            last_t = [len(md_df)] #last_t = length of md_df
            ntr_sub = range(first_t[0], last_t[0])
            ntr_sub_ranges.append(ntr_sub)
        
    ntr_df['range_ntr'] = ntr_sub_ranges
    return ntr_df #outputs df with subject, and range_ntr as the next column

In [59]:
sub_ranges = ntr_sub(df, 5, [202, 218, 222, 310, 314, 401, 403, 405, 407, 415, 204, 220, 224, 309, 313, 402, 404, 406, 408, 416, 325, 329, 326, 330])

In [60]:
sub_ranges

,subject#,range_ntr
0,1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,2,"(509, 510, 511, 512, 513, 514, 515, 516, 517, ..."
2,3,"(1110, 1111, 1112, 1113, 1114, 1115, 1116, 111..."
3,4,"(1380, 1381, 1382, 1383, 1384, 1385, 1386, 138..."
4,5,"(1866, 1867, 1868, 1869, 1870, 1871, 1872, 187..."
5,6,"(2450, 2451, 2452, 2453, 2454, 2455, 2456, 245..."
6,7,"(2851, 2852, 2853, 2854, 2855, 2856, 2857, 285..."
7,8,"(3381, 3382, 3383, 3384, 3385, 3386, 3387, 338..."
8,9,"(3737, 3738, 3739, 3740, 3741, 3742, 3743, 374..."
9,10,"(4011, 4012, 4013, 4014, 4015, 4016, 4017, 401..."


In [62]:
def md_subs(df, numsessions, subs): 
    #load model data
    md_dict = md.get_model_data(df, numsessions, subs) #model data as a dict
    md_df = pd.DataFrame.from_dict(md_dict) #model data as a df
    md_df = md_df.drop(md_df[(md_df.C == 0) & (md_df.startSubject == 0)].index) #remove non-responses, while keeping changes in startSubject
    md_df.reset_index(inplace = True)
    
    subs_unique = md_df.startSubject.unique()
    subs_unique = np.delete(subjects, 1) #remove subject 0 (not a real subject)
    
    return md_df, subs_unique

In [63]:
md_df, subs_unique = md_subs(df, 5, [202, 218, 222, 310, 314, 401, 403, 405, 407, 415, 204, 220, 224, 309, 313, 402, 404, 406, 408, 416, 325, 329, 326, 330])

In [ ]:
subs_unique

In [ ]:
ntr_df.index
range_ts = ntr_df.at[0, 'range_ntr']
md_df[range_ts[0]:(range_ts[-1]+1)]
# md_df[0:509]

In [76]:
def chosen_outcome_data(df, numsessions, subs): 
    md_dict = md.get_model_data(df, numsessions, subs) #model data as a dict
    md_df = pd.DataFrame.from_dict(md_dict) #model data as a df
    md_df = md_df.drop(md_df[(md_df.C == 0) & (md_df.startSubject == 0)].index) #remove non-responses, while keeping changes in startSubject
    md_df.reset_index(inplace = True)
    
    subs_unique = md_df.startSubject.unique()
    subs_unique = np.delete(subjects, 1) #remove subject 0 (not a real subject)
    
    for row in md_df.index: #add outcome column (where loss == 0, and win == 1)
        if md_df.at[row,'R'] == 0: #loss
            md_df.at[row,'outcome'] = 0
        if md_df.at[row,'P'] == 0: #win
            md_df.at[row,'outcome'] = 1
    
    chosen = []
    outcome = []
    ntr_df = ntr_sub(df, numsessions, subs)
    
    for row in ntr_df.index:
        range_ts = ntr_df.at[row, 'range_ntr'] #range of trials
        filtered_md_df = md_df[range_ts[0]:(range_ts[-1]+1)]
        chosen.append(filtered_md_df['O'].values)
        outcome.append(filtered_md_df['outcome'].values)
                               
    return chosen, outcome

In [77]:
chosen1, outcome1 = chosen_outcome_data(df, 5, [202, 218, 222, 310, 314, 401, 403, 405, 407, 415, 204, 220, 224, 309, 313, 402, 404, 406, 408, 416, 325, 329, 326, 330])

In [82]:
# chosen1

In [79]:
def sub_df_complete(df, numsessions, subs):
    """takes the output from md_subs, ntr_sub and chosen_outcome_data functions and outputs the complete df"""
    md_df, subs_unique = md_subs(df, numsessions, subs)
    ntr_df = ntr_sub(df, numsessions, subs)
    chosen_data, outcome_data = chosen_outcome_data(df, numsessions, subs)
    
    ntr_df['chosen_data'] = chosen_data
    ntr_df['outcome_data'] = outcome_data
    
    return ntr_df

In [80]:
sub_df = sub_df_complete(df, 5, [202, 218, 222, 310, 314, 401, 403, 405, 407, 415, 204, 220, 224, 309, 313, 402, 404, 406, 408, 416, 325, 329, 326, 330])

In [81]:
sub_df

,subject#,range_ntr,chosen_data,outcome_data
0,1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1, 1, 4, 2, 1, 3, 3, 3, 3, 1, 4, 1, 2, 3, 4, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ..."
1,2,"(509, 510, 511, 512, 513, 514, 515, 516, 517, ...","[0, 3, 3, 2, 2, 1, 4, 2, 3, 2, 2, 3, 2, 3, 2, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, ..."
2,3,"(1110, 1111, 1112, 1113, 1114, 1115, 1116, 111...","[4, 1, 2, 4, 3, 1, 1, 1, 2, 2, 4, 2, 1, 2, 1, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ..."
3,4,"(1380, 1381, 1382, 1383, 1384, 1385, 1386, 138...","[0, 1, 4, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 1, 1, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ..."
4,5,"(1866, 1867, 1868, 1869, 1870, 1871, 1872, 187...","[3, 1, 2, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
5,6,"(2450, 2451, 2452, 2453, 2454, 2455, 2456, 245...","[1, 1, 1, 1, 4, 1, 3, 4, 3, 1, 4, 1, 4, 1, 3, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
6,7,"(2851, 2852, 2853, 2854, 2855, 2856, 2857, 285...","[1, 2, 2, 2, 3, 2, 2, 2, 2, 1, 1, 2, 3, 2, 4, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
7,8,"(3381, 3382, 3383, 3384, 3385, 3386, 3387, 338...","[2, 2, 4, 2, 4, 2, 2, 2, 4, 3, 2, 2, 2, 3, 4, ...","[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, ..."
8,9,"(3737, 3738, 3739, 3740, 3741, 3742, 3743, 374...","[3, 2, 2, 2, 1, 2, 3, 1, 3, 4, 4, 1, 2, 3, 3, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, ..."
9,10,"(4011, 4012, 4013, 4014, 4015, 4016, 4017, 401...","[0, 1, 4, 3, 4, 2, 1, 1, 4, 2, 4, 2, 1, 4, 2, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."


Does len(chosen_data) match range_ntr
There's 0's in chosen_data